In [8]:
import mdtraj as md
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np
import pandas as pd

In [9]:
data_dir = '/Volumes/JGI/AAHD/round_1/'
traj_paths = [data_dir+'{}ns/100ns-production-stripped.xtc'.format(i+1) for i in range(100)]
top_path = '../data/MD/2agy_final_min-stripped_1frame.pdb'
xtal_path = '../data/MD/2agy_c36_state0.pdb'

Load the crystal structure to align to, I need to get rid of the crystal waters though. 

In [10]:
xtal = md.load(xtal_path)
xtal = xtal.atom_slice(xtal.top.select('not water'))

Load the and process data

In [11]:
alpha_ix = xtal.top.select('backbone and name CA')

In [12]:
rmsds = []
for path in traj_paths: 
    print(path)
    traj = md.load(path, top=top_path, stride=100)
    traj.superpose(reference=xtal, atom_indices = alpha_ix, ref_atom_indices = alpha_ix)
    rmsds.append(md.rmsd(target = traj, reference = xtal, atom_indices = alpha_ix))
    

/Volumes/JGI/AAHD/round_1/1ns/100ns-production-stripped.xtc
/Volumes/JGI/AAHD/round_1/2ns/100ns-production-stripped.xtc
/Volumes/JGI/AAHD/round_1/3ns/100ns-production-stripped.xtc
/Volumes/JGI/AAHD/round_1/4ns/100ns-production-stripped.xtc
/Volumes/JGI/AAHD/round_1/5ns/100ns-production-stripped.xtc
/Volumes/JGI/AAHD/round_1/6ns/100ns-production-stripped.xtc
/Volumes/JGI/AAHD/round_1/7ns/100ns-production-stripped.xtc
/Volumes/JGI/AAHD/round_1/8ns/100ns-production-stripped.xtc
/Volumes/JGI/AAHD/round_1/9ns/100ns-production-stripped.xtc
/Volumes/JGI/AAHD/round_1/10ns/100ns-production-stripped.xtc
/Volumes/JGI/AAHD/round_1/11ns/100ns-production-stripped.xtc
/Volumes/JGI/AAHD/round_1/12ns/100ns-production-stripped.xtc
/Volumes/JGI/AAHD/round_1/13ns/100ns-production-stripped.xtc
/Volumes/JGI/AAHD/round_1/14ns/100ns-production-stripped.xtc
/Volumes/JGI/AAHD/round_1/15ns/100ns-production-stripped.xtc
/Volumes/JGI/AAHD/round_1/16ns/100ns-production-stripped.xtc
/Volumes/JGI/AAHD/round_1/17ns/10

Convert to data frame and save. 

In [14]:
rmsd = np.concatenate(rmsds)
idx = np.concatenate([np.repeat(i+1, rmsds[i].shape[0]) for i in range(len(rmsds))])
pd.DataFrame({'rmsd': rmsd, 'traj_idx': idx}).to_csv('../outputs/rmsd_ca.csv', index=False)